In [4]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


# New Section

In [5]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/los.csv");
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [6]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



how='any'


In [7]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [9]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
df.na.drop(subset=['Name']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
df.na.drop(subset=['Name','Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
df.na.drop(how='all',subset=['Name','Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
df.na.drop(how='all',subset=['Name','Sales','Id']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
df.na.fill({'Name': 'Missing Name', 'Sales': 0}).show()

+----+------------+-----+
|  Id|        Name|Sales|
+----+------------+-----+
|emp1|        John|  0.0|
|emp2|Missing Name|  0.0|
|emp3|Missing Name|345.0|
|emp4|       Cindy|456.0|
+----+------------+-----+



In [15]:
from pyspark.sql.functions import *

In [16]:
ms=df.select(mean('Sales')).show()

+----------+
|avg(Sales)|
+----------+
|     400.5|
+----------+



In [17]:
ms=df.select(avg('Sales'));
ms.show()

+----------+
|avg(Sales)|
+----------+
|     400.5|
+----------+



In [18]:
ls=ms.collect();
ls

[Row(avg(Sales)=400.5)]

In [19]:
kg=(ls[0].asDict()['avg(Sales)'])

In [20]:
df.na.fill({'Sales':kg}).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
df.na.fill({'Sales':(((df.select(avg('Sales'))).collect())[0].asDict())['avg(Sales)']}).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [34]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



isNull() , isNotNull()

In [38]:
df.filter(~df['Name'].isNotNull()).show()

+----+----+-----+
|  Id|Name|Sales|
+----+----+-----+
|emp2|null| null|
|emp3|null|345.0|
+----+----+-----+



In [39]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [42]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [43]:
df.withColumn("Sales_vale_in_Integer",df['sales'].cast('int')).show()

+----+-----+-----+---------------------+
|  Id| Name|Sales|Sales_vale_in_Integer|
+----+-----+-----+---------------------+
|emp1| John| null|                 null|
|emp2| null| null|                 null|
|emp3| null|345.0|                  345|
|emp4|Cindy|456.0|                  456|
+----+-----+-----+---------------------+



In [47]:
ms=df.withColumn("Ida",df['Id'].cast('int'));
ms.filter(ms['Ida'].isNull()).show()

+----+-----+-----+----+
|  Id| Name|Sales| Ida|
+----+-----+-----+----+
|emp1| John| null|null|
|emp2| null| null|null|
|emp3| null|345.0|null|
|emp4|Cindy|456.0|null|
+----+-----+-----+----+

